# **Merging ICU Database**

**Import packages and database**

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
# Connect to your local SQLite file
conn = sqlite3.connect("eicu_v2_0_1.sqlite3")

**Read a Table**

In [ ]:
# Read a table into pandas
patient_df = pd.read_sql("SELECT * FROM patient", conn)

# Check first 5 rows
patient_df.head()

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,141764,129391,Female,87,Caucasian,59,91,,157.5,23:36:00,...,ICU to SDU,2,stepdown/other,,,18:58:00,344,Home,Alive,002-1039
1,141765,129391,Female,87,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",157.5,23:36:00,...,Emergency Department,1,admit,46.5,45,13:14:00,2250,Step-Down Unit (SDU),Alive,002-1039
2,143870,131022,Male,76,Caucasian,68,103,"Endarterectomy, carotid",167,20:46:00,...,Operating Room,1,admit,77.5,79.4,10:00:00,793,Floor,Alive,002-12289
3,144815,131736,Female,34,Caucasian,56,82,"Overdose, other toxin, poison or drug",172.7,01:44:00,...,Emergency Department,1,admit,60.3,60.7,20:48:00,1121,Other External,Alive,002-1116
4,145427,132209,Male,61,Caucasian,68,103,"GI perforation/rupture, surgery for",177.8,23:48:00,...,Operating Room,1,admit,91.7,93.1,22:47:00,1369,Floor,Alive,002-12243


## **Merging from patient table**

In [ ]:
# Pull only the columns we want from the patient table
patient_cols = [
    "patientunitstayid", "gender", "age", "ethnicity", "hospitalid",
    "unittype", "admissionheight", "admissionweight", "dischargeweight",
    "hospitaldischargestatus", "unitdischargestatus",
    "unitadmitsource", "hospitaladmitsource"]

query = f"SELECT {', '.join(patient_cols)} FROM patient"
patient_df = pd.read_sql(query, conn)

# 3Create outcome variables
# Main outcome: died during hospitalization
patient_df['bad_outcome'] = (
    patient_df['hospitaldischargestatus'].str.lower().str.contains('expired')
).astype(int)

# Secondary outcome: died in ICU
patient_df['ICU_death'] = (
    patient_df['unitdischargestatus'].str.lower().str.contains('expired')
).astype(int)

# Pull hospital info and merge
hospital_df = pd.read_sql(
    "SELECT hospitalid, numbedscategory, teachingstatus, region FROM hospital", conn)

merged_df = pd.merge(patient_df, hospital_df, on='hospitalid', how='left')

In [ ]:
merged_df.head()

,patientunitstayid,gender,age,ethnicity,hospitalid,unittype,admissionheight,admissionweight,dischargeweight,hospitaldischargestatus,unitdischargestatus,unitadmitsource,hospitaladmitsource,bad_outcome,ICU_death,numbedscategory,teachingstatus,region
0,141764,Female,87,Caucasian,59,Med-Surg ICU,157.5,,,Alive,Alive,ICU to SDU,,0,0,<100,f,Midwest
1,141765,Female,87,Caucasian,59,Med-Surg ICU,157.5,46.5,45,Alive,Alive,Emergency Department,,0,0,<100,f,Midwest
2,143870,Male,76,Caucasian,68,SICU,167,77.5,79.4,Alive,Alive,Operating Room,Operating Room,0,0,<100,f,Midwest
3,144815,Female,34,Caucasian,56,Med-Surg ICU,172.7,60.3,60.7,Alive,Alive,Emergency Department,Emergency Department,0,0,<100,f,Midwest
4,145427,Male,61,Caucasian,68,SICU,177.8,91.7,93.1,Alive,Alive,Operating Room,Emergency Department,0,0,<100,f,Midwest


In [ ]:
print("Shape:", merged_df.shape)
print("Total bad outcomes (hospital deaths):", merged_df['bad_outcome'].sum())
print("Total ICU deaths:", merged_df['ICU_death'].sum())

Shape: (2520, 18)
Total bad outcomes (hospital deaths): 212
Total ICU deaths: 126


## **Merging from apachepatientresult table**

In [ ]:
# Pull APACHE patient results
apache_cols = [
    "patientunitstayid",
    "acutephysiologyscore",
    "apachescore",
    "predictedicumortality",
    "predictedhospitalmortality",
    "actualicumortality",
    "actualhospitalmortality",
    "actualventdays",
    "actualiculos",
    "actualhospitallos"]
apache_df = pd.read_sql(
    f"SELECT {', '.join(apache_cols)} FROM apachepatientresult",
    conn)

# Aggregate APACHE results to one row per patient (use max = worst severity) - more meaningful for APACHE ICU
apache_df_unique = apache_df.groupby('patientunitstayid').agg({
    'acutephysiologyscore': 'max',
    'apachescore': 'max',
    'predictedicumortality': 'max',
    'predictedhospitalmortality': 'max',
    'actualicumortality': 'max',
    'actualhospitalmortality': 'max',
    'actualventdays': 'max',
    'actualiculos': 'max',
    'actualhospitallos': 'max'
}).reset_index()

# Merge APACHE features into merged_df
merged_df = pd.merge(merged_df, apache_df_unique, on='patientunitstayid', how='left')


In [ ]:
# Inspect final merged dataframe
merged_df.head()

,patientunitstayid,gender,age,ethnicity,hospitalid,unittype,admissionheight,admissionweight,dischargeweight,hospitaldischargestatus,...,region,acutephysiologyscore,apachescore,predictedicumortality,predictedhospitalmortality,actualicumortality,actualhospitalmortality,actualventdays,actualiculos,actualhospitallos
0,141764,Female,87,Caucasian,59,Med-Surg ICU,157.5,,,Alive,...,Midwest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141765,Female,87,Caucasian,59,Med-Surg ICU,157.5,46.5,45,Alive,...,Midwest,23.0,47.0,8.2471913877810981E-3,3.7319948856373762E-2,ALIVE,ALIVE,,1.5625,1.8222
2,143870,Male,76,Caucasian,68,SICU,167,77.5,79.4,Alive,...,Midwest,43.0,60.0,1.7738751589020281E-2,0.02893216261220858,ALIVE,ALIVE,,0.5506,0.8465
3,144815,Female,34,Caucasian,56,Med-Surg ICU,172.7,60.3,60.7,Alive,...,Midwest,25.0,25.0,2.1330459623421791E-3,4.2529613427439049E-3,ALIVE,ALIVE,,0.7784,0.8063
4,145427,Male,61,Caucasian,68,SICU,177.8,91.7,93.1,Alive,...,Midwest,26.0,37.0,9.514384325165182E-3,2.1406808566348679E-2,ALIVE,ALIVE,,0.9506,3.6618


In [ ]:
print("Shape after merging APACHE (unique):", merged_df.shape)
print("Total bad outcomes (hospital deaths):", merged_df['bad_outcome'].sum())
print("Total ICU deaths:", merged_df['ICU_death'].sum())

# Added 9 more rows! perfect

Shape after merging APACHE (unique): (2520, 27)
Total bad outcomes (hospital deaths): 212
Total ICU deaths: 126


## **Merging from apacheapsvar**

In [ ]:
# Columns to keep from apacheApsVar
aps_cols = [
    "patientunitstayid",
    "intubated",
    "vent",
    "dialysis",
    "temperature",
    "respiratoryrate",
    "heartrate",
    "meanbp",
    "urine",
    "creatinine",
    "bun",
    "sodium",
    "hematocrit",
    "albumin",
    "glucose",
    "bilirubin",
    "pao2",
    "pco2",
    "fio2",
    "ph"]

# Pull the table
aps_df = pd.read_sql(
    f"SELECT {', '.join(aps_cols)} FROM apacheapsvar",
    conn)

# Aggregate per patient — take max for interventions, worst/best for labs
aps_df_unique = aps_df.groupby('patientunitstayid').agg({
    'intubated': 'max',       # 1 if intubated at any point
    'vent': 'max',            # 1 if ventilated at any point
    'dialysis': 'max',        # 1 if on dialysis at any point
    'temperature': 'max',     # worst = highest temp (could also use min for hypothermia if relevant)
    'respiratoryrate': 'max', # max respiratory stress
    'heartrate': 'max',
    'meanbp': 'min',          # hypotension is dangerous, so min
    'urine': 'min',           # low urine = poor kidney function
    'creatinine': 'max',      # high creatinine = worse kidney health
    'bun': 'max',             # high BUN = worse kidney health
    'sodium': 'min',          # hyponatremia can be critical
    'hematocrit': 'min',      # low Hct can indicate blood loss/anemia
    'albumin': 'min',         # low albumin = poor prognosis
    'glucose': 'max',         # hyperglycemia
    'bilirubin': 'max',       # liver function
    'pao2': 'min',            # hypoxemia
    'pco2': 'max',            # hypercapnia
    'fio2': 'max',            # worst oxygen support required
    'ph': 'min'               # acidosis
}).reset_index()

# Merge into merged_df
merged_df = pd.merge(merged_df, aps_df_unique, on='patientunitstayid', how='left')


In [ ]:
# Inspect merged dataframe
merged_df.head()

,patientunitstayid,gender,age,ethnicity,hospitalid,unittype,admissionheight,admissionweight,dischargeweight,hospitaldischargestatus,...,bun,sodium,hematocrit,albumin,glucose,bilirubin,pao2,pco2,fio2,ph
0,141764,Female,87,Caucasian,59,Med-Surg ICU,157.5,,,Alive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141765,Female,87,Caucasian,59,Med-Surg ICU,157.5,46.5,45,Alive,...,28.0,139.0,37.8,-1.0,61.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,143870,Male,76,Caucasian,68,SICU,167,77.5,79.4,Alive,...,14.0,133.0,34.1,-1.0,140.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,144815,Female,34,Caucasian,56,Med-Surg ICU,172.7,60.3,60.7,Alive,...,6.0,141.0,36.6,3.6,82.0,0.5,-1.0,-1.0,-1.0,-1.0
4,145427,Male,61,Caucasian,68,SICU,177.8,91.7,93.1,Alive,...,14.0,141.0,40.4,-1.0,118.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [ ]:
print("Shape after merging apacheApsVar:", merged_df.shape)

Shape after merging apacheApsVar: (2520, 46)


## **CHECKPOINT**

In [ ]:
# List all column names
print(merged_df.columns.tolist())

['patientunitstayid', 'gender', 'age', 'ethnicity', 'hospitalid', 'unittype', 'admissionheight', 'admissionweight', 'dischargeweight', 'hospitaldischargestatus', 'unitdischargestatus', 'unitadmitsource', 'hospitaladmitsource', 'bad_outcome', 'ICU_death', 'numbedscategory', 'teachingstatus', 'region', 'acutephysiologyscore', 'apachescore', 'predictedicumortality', 'predictedhospitalmortality', 'actualicumortality', 'actualhospitalmortality', 'actualventdays', 'actualiculos', 'actualhospitallos', 'intubated', 'vent', 'dialysis', 'temperature', 'respiratoryrate', 'heartrate', 'meanbp', 'urine', 'creatinine', 'bun', 'sodium', 'hematocrit', 'albumin', 'glucose', 'bilirubin', 'pao2', 'pco2', 'fio2', 'ph']


In [ ]:
# Select columns that may leak
leak_cols = [
    'bad_outcome',
    'ICU_death',
    'actualicumortality',
    'actualhospitalmortality',
    'actualiculos',
    'actualhospitallos',
    'actualventdays',
    'predictedicumortality',
    'predictedhospitalmortality']

# Show the first few rows
merged_df[leak_cols].head(500)


,bad_outcome,ICU_death,actualicumortality,actualhospitalmortality,actualiculos,actualhospitallos,actualventdays,predictedicumortality,predictedhospitalmortality
0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,ALIVE,ALIVE,1.5625,1.8222,,8.2471913877810981E-3,3.7319948856373762E-2
2,0,0,ALIVE,ALIVE,0.5506,0.8465,,1.7738751589020281E-2,0.02893216261220858
3,0,0,ALIVE,ALIVE,0.7784,0.8063,,2.1330459623421791E-3,4.2529613427439049E-3
4,0,0,ALIVE,ALIVE,0.9506,3.6618,,9.514384325165182E-3,2.1406808566348679E-2
...,...,...,...,...,...,...,...,...,...
495,0,0,ALIVE,ALIVE,3.9652,14.2222,4.0,0.69254902679713248,0.80688466044655527
496,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
leak_cols = [
    'ICU_death',
    'actualicumortality',
    'actualhospitalmortality',
    'predictedicumortality',
    'predictedhospitalmortality']

# We want to get rid of predicted because feeding this into the model will kind of be like cheating. And then the ICU death, actual icu mortality and hospital mortality is already incorporated into our bad_outcome variable

**NOTE:** Please note that some bad_outcome's are rows with all NA's?

In [ ]:
# Drop them from features, keep only for reference/target
merged_df = merged_df.drop(columns=leak_cols)

## **Merging from apachepredsvar**

In [ ]:
predvar_cols = [
    'patientunitstayid', 'age', 'gender', 'admitSource',
    'aids', 'hepaticFailure', 'lymphoma', 'metastaticCancer',
    'leukemia', 'immunosuppression', 'cirrhosis', 'diabetes',
    'electiveSurgery', 'activeTx', 'ventDay1', 'readmit']

predvar_df = pd.read_sql(
    f"SELECT {', '.join(predvar_cols)} FROM apachepredvar",
    conn)

# Merge into merged_df
merged_df = pd.merge(merged_df, predvar_df, on='patientunitstayid', how='left')

In [ ]:
# Update bad_outcome to include readmission
merged_df['bad_outcome'] = merged_df.apply(
    lambda row: 1 if (row['bad_outcome'] == 1 or row['readmit'] == 1) else 0,
    axis=1)

In [ ]:
print("Shape after merging apachepredvar:", merged_df.shape)

Shape after merging apachepredvar: (2520, 70)


In [ ]:
# Just checking to see if it worked
meep = [
    'bad_outcome',
    'readmit']

# Show the first few rows
merged_df[meep].head(200)

# Yes it worked, delete the readmit column later

,bad_outcome,readmit
0,0,NaN
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
...,...,...
195,0,0.0
196,0,0.0
197,0,0.0
198,0,0.0


## **CHECKPOINT 2**

In [ ]:
# List all column names
print(merged_df.columns.tolist())

['patientunitstayid', 'gender_x', 'age_x', 'ethnicity', 'hospitalid', 'unittype', 'admissionheight', 'admissionweight', 'dischargeweight', 'hospitaldischargestatus', 'unitdischargestatus', 'unitadmitsource', 'hospitaladmitsource', 'bad_outcome', 'numbedscategory', 'teachingstatus', 'region', 'acutephysiologyscore', 'apachescore', 'actualventdays', 'actualiculos', 'actualhospitallos', 'intubated', 'vent', 'dialysis', 'temperature', 'respiratoryrate', 'heartrate', 'meanbp', 'urine', 'creatinine', 'bun', 'sodium', 'hematocrit', 'albumin', 'glucose', 'bilirubin', 'pao2', 'pco2', 'fio2', 'ph', 'age_y', 'gender_y', 'admitsource_x', 'aids_x', 'hepaticfailure_x', 'lymphoma_x', 'metastaticcancer_x', 'leukemia_x', 'immunosuppression_x', 'cirrhosis_x', 'diabetes_x', 'electivesurgery_x', 'activetx_x', 'ventday1_x', 'age', 'gender', 'admitsource_y', 'aids_y', 'hepaticfailure_y', 'lymphoma_y', 'metastaticcancer_y', 'leukemia_y', 'immunosuppression_y', 'cirrhosis_y', 'diabetes_y', 'electivesurgery_y'

In [ ]:
merged_df.head()

,patientunitstayid,gender_x,age_x,ethnicity,hospitalid,unittype,admissionheight,admissionweight,dischargeweight,hospitaldischargestatus,...,lymphoma_y,metastaticcancer_y,leukemia_y,immunosuppression_y,cirrhosis_y,diabetes_y,electivesurgery_y,activetx_y,ventday1_y,readmit
0,141764,Female,87,Caucasian,59,Med-Surg ICU,157.5,,,Alive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141765,Female,87,Caucasian,59,Med-Surg ICU,157.5,46.5,45,Alive,...,0.0,0.0,0.0,0.0,0.0,0.0,,0.0,0.0,0.0
2,143870,Male,76,Caucasian,68,SICU,167,77.5,79.4,Alive,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0
3,144815,Female,34,Caucasian,56,Med-Surg ICU,172.7,60.3,60.7,Alive,...,0.0,0.0,0.0,0.0,0.0,0.0,,0.0,0.0,0.0
4,145427,Male,61,Caucasian,68,SICU,177.8,91.7,93.1,Alive,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0


In [ ]:
# Duplicates
huh = [
    'age_x',
    'age_y']

# Show the first few rows
merged_df[huh].head(10)

,age_x,age_y
0,87,NaN
1,87,87
2,76,76
3,34,34
4,61,61
5,55,NaN
6,55,55
7,60,60
8,28,28
9,34,NaN


In [ ]:
# Remove duplicates (the ones with y) and rename _x

# Drop all *_y columns and readmit
cols_to_drop = [col for col in merged_df.columns if col.endswith('_y')] + ['readmit']
merged_df = merged_df.drop(columns=cols_to_drop)

# Rename *_x columns to remove suffix
merged_df = merged_df.rename(columns=lambda x: x[:-2] if x.endswith('_x') else x)

In [ ]:
print(merged_df.columns.tolist())

['patientunitstayid', 'gender', 'age', 'ethnicity', 'hospitalid', 'unittype', 'admissionheight', 'admissionweight', 'dischargeweight', 'hospitaldischargestatus', 'unitdischargestatus', 'unitadmitsource', 'hospitaladmitsource', 'bad_outcome', 'numbedscategory', 'teachingstatus', 'region', 'acutephysiologyscore', 'apachescore', 'actualventdays', 'actualiculos', 'actualhospitallos', 'intubated', 'vent', 'dialysis', 'temperature', 'respiratoryrate', 'heartrate', 'meanbp', 'urine', 'creatinine', 'bun', 'sodium', 'hematocrit', 'albumin', 'glucose', 'bilirubin', 'pao2', 'pco2', 'fio2', 'ph', 'admitsource', 'aids', 'hepaticfailure', 'lymphoma', 'metastaticcancer', 'leukemia', 'immunosuppression', 'cirrhosis', 'diabetes', 'electivesurgery', 'activetx', 'ventday1', 'age', 'gender']


In [ ]:
# Duplicates
eh = [
    'age',
    'gender']

# Show the first few rows
merged_df[eh].head(10)

,age,age,gender,gender
0,87,NaN,Female,NaN
1,87,87,Female,1.0
2,76,76,Male,0.0
3,34,34,Female,1.0
4,61,61,Male,0.0
5,55,NaN,Female,NaN
6,55,55,Female,1.0
7,60,60,Female,1.0
8,28,28,Male,0.0
9,34,NaN,Female,NaN


In [ ]:
merged_df = merged_df.iloc[:, :-2]  # keeps all columns except the last two

In [ ]:
print(merged_df.columns.tolist())

['patientunitstayid', 'gender', 'age', 'ethnicity', 'hospitalid', 'unittype', 'admissionheight', 'admissionweight', 'dischargeweight', 'hospitaldischargestatus', 'unitdischargestatus', 'unitadmitsource', 'hospitaladmitsource', 'bad_outcome', 'numbedscategory', 'teachingstatus', 'region', 'acutephysiologyscore', 'apachescore', 'actualventdays', 'actualiculos', 'actualhospitallos', 'intubated', 'vent', 'dialysis', 'temperature', 'respiratoryrate', 'heartrate', 'meanbp', 'urine', 'creatinine', 'bun', 'sodium', 'hematocrit', 'albumin', 'glucose', 'bilirubin', 'pao2', 'pco2', 'fio2', 'ph', 'admitsource', 'aids', 'hepaticfailure', 'lymphoma', 'metastaticcancer', 'leukemia', 'immunosuppression', 'cirrhosis', 'diabetes', 'electivesurgery', 'activetx', 'ventday1']


In [ ]:
print("Shape after checkpoint:", merged_df.shape)

Shape after checkpoint: (2520, 53)


## **Merging from diagnosis and admissiondx and labs and nursecharting**

Hehe we skip because it is all string and diagnosis priority does not really contribute to our research question! Also for lab, it's just text. Same with nurse charting. We need to focus more on numerical values and vitals.

## **Merging from intakeoutput**

In [ ]:
# Pull table from SQL
intakeoutput_df = pd.read_sql("SELECT patientunitstayid, nettotal FROM IntakeOutput", conn)

# Aggregate netTotal per patient
io_agg = intakeoutput_df.groupby('patientunitstayid').agg({
    'nettotal': 'sum'  # or 'last' for final recorded value
}).reset_index()


In [ ]:
# Merge into main dataframe
merged_df = pd.merge(merged_df, io_agg, on='patientunitstayid', how='left')

In [ ]:
print("Shape after merging intakeoutput:", merged_df.shape)

Shape after merging intakeoutput: (2520, 54)


In [ ]:
merged_df["nettotal"]

,nettotal
0,-500.00
1,3025.00
2,4201.00
3,4500.00
4,11283.16
...,...
2515,17294.34
2516,19745.35
2517,2299.50
2518,786.90


## **Merging from vitalperiodic**

In [ ]:
vital_cols = [
    "patientunitstayid",
    "observationoffset",
    "temperature",
    "sao2",
    "heartrate",
    "respiration",
    "systemicsystolic",
    "systemicdiastolic",
    "systemicmean"]

vital_df = pd.read_sql(
    f"SELECT {', '.join(vital_cols)} FROM vitalPeriodic",
    conn)

In [ ]:
# Columns that should be numeric
vital_numeric_cols = [
    'temperature', 'sao2', 'heartrate', 'respiration',
    'systemicsystolic', 'systemicdiastolic', 'systemicmean']

# Convert columns to numeric, invalid parsing becomes NaN
for col in vital_numeric_cols:
    vital_df[col] = pd.to_numeric(vital_df[col], errors='coerce')

# Optional: drop rows where all numeric columns are NaN
vital_df = vital_df.dropna(subset=vital_numeric_cols, how='all')

In [ ]:
vital_df.dtypes

,0
patientunitstayid,int64
observationoffset,int64
temperature,float64
sao2,float64
heartrate,float64
respiration,float64
systemicsystolic,float64
systemicdiastolic,float64
systemicmean,float64


In [ ]:
vital_df.head()

,patientunitstayid,observationoffset,temperature,sao2,heartrate,respiration,systemicsystolic,systemicdiastolic,systemicmean
0,141765,1179,NaN,NaN,82.0,NaN,NaN,NaN,NaN
1,141765,189,NaN,97.0,76.0,30.0,NaN,NaN,NaN
2,141765,1169,NaN,NaN,84.0,NaN,NaN,NaN,NaN
3,141765,1534,NaN,NaN,92.0,NaN,NaN,NaN,NaN
4,141765,1164,NaN,NaN,86.0,NaN,NaN,NaN,NaN


In [ ]:
# Aggregate first 24 hours of vital signs per patient:
# - temperature: mean, max, min → trends, fever peaks, low temps
# - saO2: mean, min → O2 drops captured
# - heartRate: mean, max, std → average, peaks, variability
# - respiration: mean, max → average and peak RR
# - BP: mean, std → baseline and variability
# Observation offsets >1440 mins are excluded to focus on early ICU vitals

# Filter first 24 hours
vital_df = vital_df[vital_df['observationoffset'] <= 1440]

# Aggregate per patient
# Note: we compute mean/max/min for some features and mean/std for others
vital_agg = vital_df.groupby('patientunitstayid').agg({
    'temperature': ['mean', 'max', 'min'],   # core temperature trends
    'sao2': ['mean', 'min'],                # O2 saturation trend and drops
    'heartrate': ['mean', 'max', 'std'],    # average, peaks, variability
    'respiration': ['mean', 'max'],         # average and peak RR
    'systemicsystolic': ['mean', 'std'],    # BP trends and variability
    'systemicdiastolic': ['mean', 'std'],
    'systemicmean': ['mean', 'std']})

In [ ]:
# Flatten multi-index columns
vital_agg.columns = ['_'.join(col) for col in vital_agg.columns]

# Reset index for merging
vital_agg = vital_agg.reset_index()

# Merge into your main dataframe
merged_df = pd.merge(merged_df, vital_agg, on='patientunitstayid', how='left')

#Now each patient has summary vital signs from the first 24 hours:
# temperature_mean, temperature_max, temperature_min, saO2_mean, saO2_min, etc.

In [ ]:
print("Shape after merging vitalperiodic:", merged_df.shape)

Shape after merging vitalperiodic: (2520, 70)


## **CHECKPOINT 3**

In [ ]:
# List all column names
print(merged_df.columns.tolist())

['patientunitstayid', 'gender', 'age', 'ethnicity', 'hospitalid', 'unittype', 'admissionheight', 'admissionweight', 'dischargeweight', 'hospitaldischargestatus', 'unitdischargestatus', 'unitadmitsource', 'hospitaladmitsource', 'bad_outcome', 'numbedscategory', 'teachingstatus', 'region', 'acutephysiologyscore', 'apachescore', 'actualventdays', 'actualiculos', 'actualhospitallos', 'intubated', 'vent', 'dialysis', 'temperature', 'respiratoryrate', 'heartrate', 'meanbp', 'urine', 'creatinine', 'bun', 'sodium', 'hematocrit', 'albumin', 'glucose', 'bilirubin', 'pao2', 'pco2', 'fio2', 'ph', 'admitsource', 'aids', 'hepaticfailure', 'lymphoma', 'metastaticcancer', 'leukemia', 'immunosuppression', 'cirrhosis', 'diabetes', 'electivesurgery', 'activetx', 'ventday1', 'nettotal', 'temperature_mean', 'temperature_max', 'temperature_min', 'sao2_mean', 'sao2_min', 'heartrate_mean', 'heartrate_max', 'heartrate_std', 'respiration_mean', 'respiration_max', 'systemicsystolic_mean', 'systemicsystolic_std',

In [ ]:
# Check for duplicates
# Check for duplicate rows (entire row identical)
duplicate_rows = merged_df[merged_df.duplicated()]
print(f"Number of completely duplicated rows: {len(duplicate_rows)}")

# Check for duplicate patient IDs (should be 1 row per patient)
duplicate_ids = merged_df[merged_df.duplicated(subset=['patientunitstayid'])]
print(f"Number of duplicate patient IDs: {len(duplicate_ids)}")

Number of completely duplicated rows: 0
Number of duplicate patient IDs: 0


In [ ]:
# Check for redundant columns (highly similar / exact duplicates)
# For simplicity, check columns with identical values
redundant_cols = []
cols = merged_df.columns
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        if merged_df[cols[i]].equals(merged_df[cols[j]]):
            redundant_cols.append((cols[i], cols[j]))
print("Redundant columns (identical values):", redundant_cols)

Redundant columns (identical values): [('vent', 'ventday1')]


In [ ]:
merged_df = merged_df.drop(columns=['vent'])

In [ ]:
# Check missingness per column
missing_perc = merged_df.isna().mean() * 100
missing_summary = missing_perc[missing_perc > 0].sort_values(ascending=False)
print("Columns with missing values (%):")
print(missing_summary)

Columns with missing values (%):
temperature_min           93.968254
temperature_mean          93.968254
temperature_max           93.968254
systemicsystolic_std      84.325397
systemicdiastolic_std     84.325397
systemicmean_std          84.246032
systemicdiastolic_mean    84.166667
systemicsystolic_mean     84.126984
systemicmean_mean         84.047619
acutephysiologyscore      27.063492
actualventdays            27.063492
apachescore               27.063492
actualhospitallos         27.063492
actualiculos              27.063492
respiration_mean          14.563492
respiration_max           14.563492
nettotal                  13.293651
intubated                 12.500000
dialysis                  12.500000
temperature               12.500000
pao2                      12.500000
bilirubin                 12.500000
glucose                   12.500000
albumin                   12.500000
hematocrit                12.500000
sodium                    12.500000
bun                       12.50

In [ ]:
# List of columns to drop due to extremely high missingness
high_missing_cols = [
    'temperature_min', 'temperature_mean', 'temperature_max',
    'systemicsystolic_std', 'systemicdiastolic_std', 'systemicmean_std',
    'systemicdiastolic_mean', 'systemicsystolic_mean', 'systemicmean_mean']

# Drop these columns
merged_df_clean = merged_df.drop(columns=high_missing_cols)

Index(['patientunitstayid', 'gender', 'age', 'ethnicity', 'hospitalid',
       'unittype', 'admissionheight', 'admissionweight', 'dischargeweight',
       'hospitaldischargestatus', 'unitdischargestatus', 'unitadmitsource',
       'hospitaladmitsource', 'bad_outcome', 'numbedscategory',
       'teachingstatus', 'region', 'acutephysiologyscore', 'apachescore',
       'actualventdays', 'actualiculos', 'actualhospitallos', 'intubated',
       'dialysis', 'temperature', 'respiratoryrate', 'heartrate', 'meanbp',
       'urine', 'creatinine', 'bun', 'sodium', 'hematocrit', 'albumin',
       'glucose', 'bilirubin', 'pao2', 'pco2', 'fio2', 'ph', 'admitsource',
       'aids', 'hepaticfailure', 'lymphoma', 'metastaticcancer', 'leukemia',
       'immunosuppression', 'cirrhosis', 'diabetes', 'electivesurgery',
       'activetx', 'ventday1', 'nettotal', 'sao2_mean', 'sao2_min',
       'heartrate_mean', 'heartrate_max', 'heartrate_std', 'respiration_mean',
       'respiration_max'],
      dtype='obj

In [ ]:
merged_df_clean.shape

(2520, 60)

In [ ]:
# Move 'bad_outcome' to the end
cols = [c for c in merged_df_clean.columns if c != 'bad_outcome'] + ['bad_outcome']
merged_df_clean = merged_df_clean[cols]

# Quick check
print(merged_df_clean.columns)


Index(['patientunitstayid', 'gender', 'age', 'ethnicity', 'hospitalid',
       'unittype', 'admissionheight', 'admissionweight', 'dischargeweight',
       'hospitaldischargestatus', 'unitdischargestatus', 'unitadmitsource',
       'hospitaladmitsource', 'numbedscategory', 'teachingstatus', 'region',
       'acutephysiologyscore', 'apachescore', 'actualventdays', 'actualiculos',
       'actualhospitallos', 'intubated', 'dialysis', 'temperature',
       'respiratoryrate', 'heartrate', 'meanbp', 'urine', 'creatinine', 'bun',
       'sodium', 'hematocrit', 'albumin', 'glucose', 'bilirubin', 'pao2',
       'pco2', 'fio2', 'ph', 'admitsource', 'aids', 'hepaticfailure',
       'lymphoma', 'metastaticcancer', 'leukemia', 'immunosuppression',
       'cirrhosis', 'diabetes', 'electivesurgery', 'activetx', 'ventday1',
       'nettotal', 'sao2_mean', 'sao2_min', 'heartrate_mean', 'heartrate_max',
       'heartrate_std', 'respiration_mean', 'respiration_max', 'bad_outcome'],
      dtype='object')


In [ ]:
# Take out features that leaks
cols_to_drop = [
    'hospitaldischargestatus',
    'unitdischargestatus',
    'actualventdays',
    'actualiculos',
    'actualhospitallos']

merged_df_clean = merged_df_clean.drop(columns=cols_to_drop)

In [ ]:
# Quick check
print(merged_df_clean.columns)

Index(['patientunitstayid', 'gender', 'age', 'ethnicity', 'hospitalid',
       'unittype', 'admissionheight', 'admissionweight', 'dischargeweight',
       'unitadmitsource', 'hospitaladmitsource', 'numbedscategory',
       'teachingstatus', 'region', 'acutephysiologyscore', 'apachescore',
       'intubated', 'dialysis', 'temperature', 'respiratoryrate', 'heartrate',
       'meanbp', 'urine', 'creatinine', 'bun', 'sodium', 'hematocrit',
       'albumin', 'glucose', 'bilirubin', 'pao2', 'pco2', 'fio2', 'ph',
       'admitsource', 'aids', 'hepaticfailure', 'lymphoma', 'metastaticcancer',
       'leukemia', 'immunosuppression', 'cirrhosis', 'diabetes',
       'electivesurgery', 'activetx', 'ventday1', 'nettotal', 'sao2_mean',
       'sao2_min', 'heartrate_mean', 'heartrate_max', 'heartrate_std',
       'respiration_mean', 'respiration_max', 'bad_outcome'],
      dtype='object')


In [ ]:
# Save to CSV
merged_df_clean.to_csv("mergeICU_db.csv", index=False)


**Note:** Anything related to care plan is not included because it is post-admission documentation and reflects the state the patient is in if there is a careplan in the first place. As for physician specialty, it is not included because patients can have multiple physicians and there is no real way to aggreggate it. Additionally, one speciality can tell the model that this specific patient will most likely have this type of result if this physician sees them, introducing bias, which we are not working with anymore. Remember the research question is now:
- “Can we predict whether a critically ill patient will experience a bad outcome (death or readmission) using data from their first 24 hours in the ICU?”